Podklady pro články Anety Martínkové na iROZHLAS.cz v prosinci 2023.

In [1]:
import os
import pandas as pd

In [2]:
from src.tabulka import tabulka
from src.graf import graf

In [3]:
df = pd.read_csv(os.path.join("data","CZ_RADIO_TOP_100_tydenni.csv"))
wikid = pd.read_json(os.path.join("wikidata","wikidata.json"))
df = df.merge(wikid, on="interpret_hlavni", how="outer")

In [4]:
df = df.sort_values(by=['rok','tyden'])

## Podíl žen na sólovém interpretstvu

In [5]:
roky = [int(x) for x in df['rok'].drop_duplicates().dropna().tolist()]

In [6]:
roky

[2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022,
 2023]

In [7]:
pocet_lidi = len(df[(df['velikost'] == 'člověk') & (~df['gender'].isnull())]['interpret_hlavni'].drop_duplicates())

In [8]:
df[(df['velikost'] == 'člověk') & (~df['gender'].isin(['muž','žena']))][['interpret_hlavni','gender']].drop_duplicates()

,interpret_hlavni,gender
38680,Miley Cyrus,genderfluid
60069,Demi Lovato,genderfluid
62595,Sam Smith,nebinární
91110,Lil Uzi Vert,nebinární


In [9]:
podil_zen = pd.Series(df[(df['gender'] == 'žena')].groupby('rok').size() / df[df['gender'].isin(['žena','muž','genderfluid','nebinární'])].groupby('rok').size(), name="podíl žen").reindex(roky)

In [10]:
podil_muzu = pd.Series(df[(df['gender'] == 'muž')].groupby('rok').size() / df[df['gender'].isin(['žena','muž','genderfluid','nebinární'])].groupby('rok').size(), name="podíl mužů").reindex(roky)

In [11]:
podil_ostatnich = pd.Series(df[(df['gender'].isin(['genderfluid','nebinární']))].groupby('rok').size() / df[df['gender'].isin(['žena','muž','genderfluid','nebinární'])].groupby('rok').size(), name="podíl genderfluid a nebinárních osob").reindex(roky)

In [12]:
graf(sloupcovy=[podil_ostatnich,podil_muzu,podil_zen], target="genderovy_podil", titulek='Podíl sólově vystupujících žen a mužů v týdenních žebříčcích 100 nejhranějších písní', podtitulek=f'''Z {pocet_lidi} lidí v datech se jinak než jako žena či muž identifikují Miley Cyrus, Demi Lovato, Sam Smith a Lil Uzi Vert.''', kredity=["Zdroj dat: Česká národní skupina Mezinárodní federace hudebního průmyslu a Wikidata | Vizualizace: Michal Kašpárek, iROZHLAS.cz","https://www.irozhlas.cz/zpravy-tag/datova-zurnalistika"], procenta=[podil_ostatnich,podil_muzu,podil_zen], prvni=True, osay=' ', osay2=' ', skladany=True, naopak=False)

Graf uložen.


## Věk mužů a žen: čísla

In [24]:
df[df['gender'] == 'žena']['vek'].median()

27.0

In [25]:
df[df['gender'] == 'žena']['vek'].mean()

29.101907157078486

In [26]:
df[df['gender'] == 'muž']['vek'].median()

31.0

In [27]:
df[df['gender'] == 'muž']['vek'].mean()

33.245962324984326

## Věk mužů a žen: graf

In [13]:
df['vek'] = df['rok'] - df['datum_narozeni']

In [14]:
veky = [x for x in range(15,66)]

In [15]:
veky_z = pd.Series(df[df['gender'] == 'žena'].groupby('vek').size(),name='věky žen').reindex(veky)

In [16]:
veky_m = pd.Series(df[df['gender'] == 'muž'].groupby('vek').size(),name='věky mužů').reindex(veky)

In [17]:
graf(sloupcovy=[veky_z,veky_m], target="histogram_stari", titulek='Věky sólově vystupujících mužů a žen v týdenních žebříčcích 100 nejhranějších písní', podtitulek=f'''Do součtů počítáme každou píseň a každý týden v hitparádě zvlášť. Pro přehlednost ořezáváme sporadicky osídlené okraje grafu, tedy interprety a interpretky pod 15 a nad 65 let.''', kredity=["Zdroj dat: Česká národní skupina Mezinárodní federace hudebního průmyslu a Wikidata | Vizualizace: Michal Kašpárek, iROZHLAS.cz","https://www.irozhlas.cz/zpravy-tag/datova-zurnalistika"], histogram=True, prvni=True, osay=' ', osay2=' ', skladany=False, naopak=False, barvy=["#fd7f6f","#7eb0d5"])

Graf uložen.


## Úspěšné ženy

In [18]:
top10zen = pd.DataFrame(df[(df['gender'] == 'žena') & (df['pozice'] == 1)].groupby("interpret_hlavni")['week'].nunique().nlargest(10)).reset_index().rename(columns={'interpret_hlavni':'Interpretka','week':'Týdnů jako #1'})

In [19]:
df[(df['gender'] == 'žena') & (df['zeme_puvodu'] == "Česko")].groupby("interpret_hlavni")['track'].nunique().nlargest(10)

interpret_hlavni
Ewa Farna             22
Lucie Vondráčková     18
Lucie Bílá            16
Anna K.               15
Lenny                 15
Aneta Langerová       12
Tereza Kerndlová      11
Olga Lounová           9
Markéta Konvičková     7
Emma Smetana           6
Name: track, dtype: int64

In [20]:
df[(df['gender'] == 'žena') & (df['pozice'] == 1) & (df['zeme_puvodu'] == "Česko")].groupby("interpret_hlavni")['week'].nunique().nlargest(10)

interpret_hlavni
Lenny            4
Ewa Farna        2
Helena Zeťová    1
Name: week, dtype: int64

In [21]:
tabulka(top10zen, titulek="Ženy, které strávily nejvíce týdnů na vrcholu české hitparády", podtitulek="Z Češek se to od ledna 2006 do listopadu 2023 podařilo pouze Lenny (4 týdny), Ewě Farne (2 týdny) a Heleně Zeťové (1 týden).", poradi=True)

'<table class="dataframe table table--responsive table--w100p table--striped-red table--plain"><caption>Ženy, které strávily nejvíce týdnů na vrcholu české hitparády</caption><thead><tr style="text-align: right;"><th class="text-nowrap"></th><th class="text-nowrap">Interpretka</th><th class="text-nowrap">Týdnů jako #1</th></tr></thead><tbody class="text-sm"><tr><td>1.</td><td>Adele</td><td>24</td></tr><tr><td>2.</td><td>Nelly Furtado</td><td>24</td></tr><tr><td>3.</td><td>P!Nk</td><td>23</td></tr><tr><td>4.</td><td>Shakira</td><td>16</td></tr><tr><td>5.</td><td>Ava Max</td><td>12</td></tr><tr><td>6.</td><td>Katy Perry</td><td>12</td></tr><tr><td>7.</td><td>Alesha Dixon</td><td>11</td></tr><tr><td>8.</td><td>Rihanna</td><td>10</td></tr><tr><td>9.</td><td>Carly Rae Jepsen</td><td>8</td></tr><tr><td>10.</td><td>Lady Gaga</td><td>8</td></tr></tbody><tfoot><tr style="text-align: center;"><td colspan="100%">Z Češek se to od ledna 2006 do listopadu 2023 podařilo pouze Lenny (4 týdny), Ewě Far

In [22]:
top10cesek = pd.DataFrame(df[(df['gender'] == 'žena') & (df['zeme_puvodu'] == "Česko")].groupby("interpret_hlavni")['week'].nunique().nlargest(10)).reset_index().rename(columns={'interpret_hlavni':'Interpretka','week':'Týdnů v top 100'})

In [23]:
tabulka(top10cesek, titulek="Češky, které se v top 100 držely nejvíce týdnů", podtitulek="Data od ledna 2006 do listopadu 2023.", poradi=True)

'<table class="dataframe table table--responsive table--w100p table--striped-red table--plain"><caption>Češky, které se v top 100 držely nejvíce týdnů</caption><thead><tr style="text-align: right;"><th class="text-nowrap"></th><th class="text-nowrap">Interpretka</th><th class="text-nowrap">Týdnů v top 100</th></tr></thead><tbody class="text-sm"><tr><td>1.</td><td>Anna K.</td><td>386</td></tr><tr><td>2.</td><td>Ewa Farna</td><td>303</td></tr><tr><td>3.</td><td>Lenny</td><td>284</td></tr><tr><td>4.</td><td>Lucie Vondráčková</td><td>228</td></tr><tr><td>5.</td><td>Lucie Bílá</td><td>220</td></tr><tr><td>6.</td><td>Aneta Langerová</td><td>197</td></tr><tr><td>7.</td><td>Olga Lounová</td><td>196</td></tr><tr><td>8.</td><td>Tereza Kerndlová</td><td>139</td></tr><tr><td>9.</td><td>Helena Zeťová</td><td>107</td></tr><tr><td>10.</td><td>Lenka Filipová</td><td>104</td></tr></tbody><tfoot><tr style="text-align: center;"><td colspan="100%">Data od ledna 2006 do listopadu 2023.</td></tr></tfoot></t